In [17]:
import pandas as pd
import sys
import os
from datetime import date
import numpy as np

from dependencies.rpc import RPC

# Use the full path directly
secrets_dir = r'C:\Users\kgeoghegan\Dev\new-firmwide-report\.beacon'

beacon_rpc = RPC(secrets_dir=secrets_dir)

In [18]:
rpt_date = '20250422'

response = beacon_rpc.get_rpc('users/galaxy/jc/beacon_api/generate_gre', 
                                    rpt_date=rpt_date)

res_dict = response.json()

columns = res_dict['content'][0]
rows = res_dict['content'][1:]

df = pd.DataFrame(rows, columns=columns)

In [19]:
df

,path,Business(L0),Desk(L1),Pod(L2),Book(L3),Strategy(L4),PositionBlock(L5),Desk Strategy,Underlier,Expiry,...,$Vega,$Theta,ImpliedVolatility,IsNewTrade,Price,Type,FXRate,Type_start,Price Change,Price Change (%)
0,.Trading.Benchmark1.Crypto....BTC..BTC,,Trading,Benchmark1,Crypto,,,,BTC,,...,0.0,0.0,0.0,0.0,91181.914666,CryptoPosition,1.0,CryptoPosition,3911.055194,4.481513
1,.Trading.Benchmark1.Crypto....ETH..ETH,,Trading,Benchmark1,Crypto,,,,ETH,,...,0.0,0.0,0.0,0.0,1696.741300,CryptoPosition,1.0,CryptoPosition,121.265295,7.697058
2,.Trading.Benchmark1.Crypto....SOL..SOL,,Trading,Benchmark1,Crypto,,,,SOL,,...,0.0,0.0,0.0,0.0,144.613510,CryptoPosition,1.0,CryptoPosition,7.406330,5.397917
3,Placeholder.Placeholder.Lending.Placeholder.Pl...,Placeholder,Placeholder,Lending,Placeholder,Placeholder,Placeholder,,BTC,,...,0.0,0.0,0.0,0.0,91181.914666,CryptoPosition,1.0,CryptoPosition,3911.055194,4.481513
4,PrincipalTrading.FTXRecovery.Payment_FTX_Claim...,PrincipalTrading,FTXRecovery,Payment_FTX_Claim,,,,,BTC,,...,0.0,0.0,0.0,0.0,91181.914666,CryptoPosition,1.0,CryptoPosition,3911.055194,4.481513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,PrincipalTrading.Trading.Ventures.Perp....ENA....,PrincipalTrading,Trading,Ventures,Perp,,,,ENA,,...,0.0,0.0,0.0,0.0,0.309649,CryptoPosition,1.0,CryptoPosition,0.019817,6.837409
763,PrincipalTrading.Trading.Ventures.Perp....ETH....,PrincipalTrading,Trading,Ventures,Perp,,,,ETH,,...,0.0,0.0,0.0,0.0,1696.741300,CryptoPosition,1.0,CryptoPosition,121.265295,7.697058
764,PrincipalTrading.Trading.Ventures.Perp....PYTH...,PrincipalTrading,Trading,Ventures,Perp,,,,PYTH,,...,0.0,0.0,0.0,0.0,0.145670,CryptoPosition,1.0,CryptoPosition,0.004735,3.359705
765,PrincipalTrading.Trading.Ventures.Perp....TIA....,PrincipalTrading,Trading,Ventures,Perp,,,,TIA,,...,0.0,0.0,0.0,0.0,2.660000,CryptoPosition,1.0,CryptoPosition,0.100000,3.906250


In [21]:
df.to_csv('../output/testing/gre_output.csv')

In [ ]:
# First, import the GDT data manager
from dependencies.GDT_processor import GDTDataManager
import pytz
import pandas as pd
from datetime import datetime, timedelta

# Initialize GDT data manager and get both current and previous week data
try:
    gdt_manager = GDTDataManager()
    
    # Monkey patch the _identify_key_metrics method to handle tz-naive timestamps
    original_identify_key_metrics = gdt_manager._identify_key_metrics
    def patched_identify_key_metrics(df):
        if df.empty:
            return {}
            
        key_metrics = {}
        
        # Handle the timestamp with proper timezone awareness
        if 'Timestamp__UTC' in df.columns:
            utc_time = df['Timestamp__UTC'].iloc[0]
            # Make timezone aware if it's not already
            if utc_time.tzinfo is None:
                utc_time = utc_time.replace(tzinfo=pytz.UTC)
            est_time = utc_time.astimezone(pytz.timezone('US/Eastern'))
            key_metrics['Timestamp_EST'] = est_time
        
        # Continue with the rest of the original method's logic
        # Spot prices
        spot_metrics = ['BTC_Spot', 'ETH_Spot']
        for col in spot_metrics:
            if col in df.columns:
                key_metrics[col] = df[col].iloc[0]
        
        # SOL data
        if 'SOL_BS_Delta' in df.columns:
            key_metrics['SOL_Delta'] = df['SOL_BS_Delta'].iloc[0]
        
        # Alt data  
        if 'Alt_Net_Delta' in df.columns:
            key_metrics['Alt_Delta'] = df['Alt_Net_Delta'].iloc[0]
        
        if 'Largest_Alt' in df.columns and 'Largest_Alt_Delta' in df.columns:
            key_metrics['Largest_Alt'] = df['Largest_Alt'].iloc[0]
            key_metrics['Largest_Alt_Delta'] = df['Largest_Alt_Delta'].iloc[0]
        
        # BTC Greeks
        btc_greek_mapping = {
            'Delta': 'BTC_BS_Delta_Net',
            'Gamma': 'BTC_Smile_Gamma',
            'Vega': 'BTC_BS_Vega',
            'Theta': 'BTC_BS_Theta'
        }
        
        for greek, column in btc_greek_mapping.items():
            if column in df.columns:
                key_metrics[f'BTC_{greek}'] = df[column].iloc[0]
        
        # ETH Greeks
        eth_greek_mapping = {
            'Delta': 'ETH_BS_Delta_Net',
            'Gamma': 'ETH_Smile_Gamma',
            'Vega': 'ETH_BS_Vega',
            'Theta': 'ETH_BS_Theta'
        }
        
        for greek, column in eth_greek_mapping.items():
            if column in df.columns:
                key_metrics[f'ETH_{greek}'] = df[column].iloc[0]
                
        # Calculate total Delta, Gamma, Vega, Theta across assets
        # Start with BTC and ETH values
        total_delta = key_metrics.get('BTC_Delta', 0) + key_metrics.get('ETH_Delta', 0)
        total_gamma = key_metrics.get('BTC_Gamma', 0) + key_metrics.get('ETH_Gamma', 0)
        total_vega = key_metrics.get('BTC_Vega', 0) + key_metrics.get('ETH_Vega', 0)
        total_theta = key_metrics.get('BTC_Theta', 0) + key_metrics.get('ETH_Theta', 0)
        
        # Add SOL and Alt deltas if available
        if 'SOL_Delta' in key_metrics:
            total_delta += key_metrics['SOL_Delta']
        
        if 'Alt_Delta' in key_metrics:
            total_delta += key_metrics['Alt_Delta']
        
        # Add totals to key metrics
        key_metrics['Total_Delta'] = total_delta
        key_metrics['Total_Gamma'] = total_gamma
        key_metrics['Total_Vega'] = total_vega
        key_metrics['Total_Theta'] = total_theta
            
        return key_metrics
    
    # Replace the method with our patched version
    gdt_manager._identify_key_metrics = patched_identify_key_metrics
    
    # Get current data
    current_gdt_data, current_date = gdt_manager.get_latest_market_close()
    current_gdt_metrics = gdt_manager._identify_key_metrics(current_gdt_data)
    
    # Get data from 7 days ago
    prev_date = current_date - timedelta(days=7)
    prev_gdt_data = gdt_manager.get_market_close_data(prev_date)
    prev_gdt_metrics = gdt_manager._identify_key_metrics(prev_gdt_data)
    
except Exception as e:
    print(f"Error getting GDT data: {str(e)}")
    # Create empty dictionaries if there's an error
    current_gdt_metrics = {}
    prev_gdt_metrics = {}

# Get current data from Beacon API
# Replace the Beacon API section with this fixed version:
try:
    # Use the existing df variable instead of creating current_df
    current_df = df  # Use the already loaded dataframe
    
    # Fetch data from 7 days ago
    prev_rpt_date = (datetime.now() - timedelta(days=7)).strftime('%Y%m%d')
    prev_response = beacon_rpc.get_rpc('users/galaxy/jc/beacon_api/generate_gre', rpt_date=prev_rpt_date)
    prev_res_dict = prev_response.json()
    prev_columns = prev_res_dict['content'][0]
    prev_rows = prev_res_dict['content'][1:]
    prev_df = pd.DataFrame(prev_rows, columns=prev_columns)
    
except Exception as e:
    print(f"Error getting Beacon API data: {str(e)}")
    # If there's an error, use the current df for both
    prev_df = df.copy()

# Create a new column for categorizing underliers with special handling
def categorize_underlier(row):
    underlier = row['Underlier']
    trader = row['Pod(L2)']
    
    # Check for Passive Beta category first (applies only to Novo)
    if trader == 'Novo' and underlier in ['BGCI', 'GDAM1']:
        return 'Passive Beta'
        
    # BTC category includes these additional tickers
    if underlier in ['BTC', 'IBIT', 'FBTC', 'WBTC', 'BTCO', 'ARKB', 'GBTC', 'BTC_MT_GOX', 'BTC_FUND']:
        return 'BTC'
    # ETH category includes these additional tickers
    elif underlier in ['ETH', 'QETH', 'ETH_FUND']:
        return 'ETH'
    # SOL category includes these additional tickers
    elif underlier in ['SOL', 'FTX_SOL', 'NONCPO_SOL', 'LOCKED_SOL']:
        return 'SOL'
    # All other tickers are categorized as Alts
    else:
        return 'Alts'

# Process current data
trader_list = ['Beimnet', 'Felman', 'Bouchra']
trader_df = current_df[current_df['Pod(L2)'].isin(trader_list)].copy()
trader_df['Category'] = trader_df.apply(categorize_underlier, axis=1)

# Ensure numeric columns are properly formatted
numeric_cols = ['$Delta', '$Gamma', '$Vega', '$Theta']
for col in numeric_cols:
    trader_df[col] = pd.to_numeric(trader_df[col], errors='coerce')

# Get Novo's data
novo_df = current_df[current_df['Pod(L2)'] == 'Novo'].copy() if 'Novo' in current_df['Pod(L2)'].values else pd.DataFrame()
if not novo_df.empty:
    novo_df['Category'] = novo_df.apply(categorize_underlier, axis=1)
    for col in numeric_cols:
        novo_df[col] = pd.to_numeric(novo_df[col], errors='coerce')

# Process previous week data
prev_trader_df = prev_df[prev_df['Pod(L2)'].isin(trader_list)].copy()
prev_trader_df['Category'] = prev_trader_df.apply(categorize_underlier, axis=1)

for col in numeric_cols:
    prev_trader_df[col] = pd.to_numeric(prev_trader_df[col], errors='coerce')

# Get Novo's previous data
prev_novo_df = prev_df[prev_df['Pod(L2)'] == 'Novo'].copy() if 'Novo' in prev_df['Pod(L2)'].values else pd.DataFrame()
if not prev_novo_df.empty:
    prev_novo_df['Category'] = prev_novo_df.apply(categorize_underlier, axis=1)
    for col in numeric_cols:
        prev_novo_df[col] = pd.to_numeric(prev_novo_df[col], errors='coerce')

# Group current data
gdlp_ex_novo_summary = trader_df.groupby(['Category']).agg({
    '$Delta': 'sum',
    '$Gamma': 'sum',
    '$Vega': 'sum',
    '$Theta': 'sum'
}).reset_index()

# Group current Novo data
if not novo_df.empty:
    novo_summary = novo_df.groupby(['Category']).agg({
        '$Delta': 'sum',
        '$Gamma': 'sum',
        '$Vega': 'sum',
        '$Theta': 'sum'
    }).reset_index()
else:
    novo_summary = pd.DataFrame({
        'Category': ['BTC', 'ETH', 'SOL', 'Alts', 'Passive Beta'],
        '$Delta': [0, 0, 0, 0, 0],
        '$Gamma': [0, 0, 0, 0, 0],
        '$Vega': [0, 0, 0, 0, 0],
        '$Theta': [0, 0, 0, 0, 0]
    })

# Group previous data
prev_gdlp_ex_novo_summary = prev_trader_df.groupby(['Category']).agg({
    '$Delta': 'sum',
    '$Gamma': 'sum',
    '$Vega': 'sum',
    '$Theta': 'sum'
}).reset_index()

# Group previous Novo data
if not prev_novo_df.empty:
    prev_novo_summary = prev_novo_df.groupby(['Category']).agg({
        '$Delta': 'sum',
        '$Gamma': 'sum',
        '$Vega': 'sum',
        '$Theta': 'sum'
    }).reset_index()
else:
    prev_novo_summary = pd.DataFrame({
        'Category': ['BTC', 'ETH', 'SOL', 'Alts', 'Passive Beta'],
        '$Delta': [0, 0, 0, 0, 0],
        '$Gamma': [0, 0, 0, 0, 0],
        '$Vega': [0, 0, 0, 0, 0],
        '$Theta': [0, 0, 0, 0, 0]
    })

# Add GDLP (all traders including Novo) - current data
all_traders_df = pd.concat([trader_df, novo_df]) if not novo_df.empty else trader_df
gdlp_summary = all_traders_df.groupby(['Category']).agg({
    '$Delta': 'sum',
    '$Gamma': 'sum',
    '$Vega': 'sum',
    '$Theta': 'sum'
}).reset_index()

# Add GDLP (all traders including Novo) - previous data
prev_all_traders_df = pd.concat([prev_trader_df, prev_novo_df]) if not prev_novo_df.empty else prev_trader_df
prev_gdlp_summary = prev_all_traders_df.groupby(['Category']).agg({
    '$Delta': 'sum',
    '$Gamma': 'sum',
    '$Vega': 'sum',
    '$Theta': 'sum'
}).reset_index()

# Create a flattened dictionary for Excel mapping - current data
excel_data = {}

# Add individual trader data - current
for trader in trader_list:
    trader_data = trader_df[trader_df['Pod(L2)'] == trader]
    for category in ['BTC', 'ETH', 'SOL', 'Alts']:
        category_data = trader_data[trader_data['Category'] == category]
        delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
        gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
        vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
        theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
        
        excel_data[f'{trader} {category} Delta'] = delta_sum
        excel_data[f'{trader} {category} Gamma'] = gamma_sum
        excel_data[f'{trader} {category} Vega'] = vega_sum
        excel_data[f'{trader} {category} Theta'] = theta_sum

# Add Novo data for all categories including Passive Beta - current
if not novo_df.empty:
    for category in ['BTC', 'ETH', 'SOL', 'Alts', 'Passive Beta']:
        category_data = novo_df[novo_df['Category'] == category]
        delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
        gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
        vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
        theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
        
        excel_data[f'Novo {category} Delta'] = delta_sum
        excel_data[f'Novo {category} Gamma'] = gamma_sum
        excel_data[f'Novo {category} Vega'] = vega_sum
        excel_data[f'Novo {category} Theta'] = theta_sum

# Add GDLP ex-Novo summary - current
for category in ['BTC', 'ETH', 'SOL', 'Alts']:
    category_data = gdlp_ex_novo_summary[gdlp_ex_novo_summary['Category'] == category]
    delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
    gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
    vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
    theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
    
    excel_data[f'GDLP ex-Novo {category} Delta'] = delta_sum
    excel_data[f'GDLP ex-Novo {category} Gamma'] = gamma_sum
    excel_data[f'GDLP ex-Novo {category} Vega'] = vega_sum
    excel_data[f'GDLP ex-Novo {category} Theta'] = theta_sum

# Add GDLP summary (all traders including Novo) - current
for category in ['BTC', 'ETH', 'SOL', 'Alts', 'Passive Beta']:
    category_data = gdlp_summary[gdlp_summary['Category'] == category]
    delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
    gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
    vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
    theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
    
    excel_data[f'GDLP {category} Delta'] = delta_sum
    excel_data[f'GDLP {category} Gamma'] = gamma_sum
    excel_data[f'GDLP {category} Vega'] = vega_sum
    excel_data[f'GDLP {category} Theta'] = theta_sum

# Add individual trader data - previous
for trader in trader_list:
    trader_data = prev_trader_df[prev_trader_df['Pod(L2)'] == trader]
    for category in ['BTC', 'ETH', 'SOL', 'Alts']:
        category_data = trader_data[trader_data['Category'] == category]
        delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
        gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
        vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
        theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
        
        excel_data[f'Prev {trader} {category} Delta'] = delta_sum
        excel_data[f'Prev {trader} {category} Gamma'] = gamma_sum
        excel_data[f'Prev {trader} {category} Vega'] = vega_sum
        excel_data[f'Prev {trader} {category} Theta'] = theta_sum

# Add Novo data for all categories including Passive Beta - previous
if not prev_novo_df.empty:
    for category in ['BTC', 'ETH', 'SOL', 'Alts', 'Passive Beta']:
        category_data = prev_novo_df[prev_novo_df['Category'] == category]
        delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
        gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
        vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
        theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
        
        excel_data[f'Prev Novo {category} Delta'] = delta_sum
        excel_data[f'Prev Novo {category} Gamma'] = gamma_sum
        excel_data[f'Prev Novo {category} Vega'] = vega_sum
        excel_data[f'Prev Novo {category} Theta'] = theta_sum

# Add GDLP ex-Novo summary - previous
for category in ['BTC', 'ETH', 'SOL', 'Alts']:
    category_data = prev_gdlp_ex_novo_summary[prev_gdlp_ex_novo_summary['Category'] == category]
    delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
    gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
    vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
    theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
    
    excel_data[f'Prev GDLP ex-Novo {category} Delta'] = delta_sum
    excel_data[f'Prev GDLP ex-Novo {category} Gamma'] = gamma_sum
    excel_data[f'Prev GDLP ex-Novo {category} Vega'] = vega_sum
    excel_data[f'Prev GDLP ex-Novo {category} Theta'] = theta_sum

# Add GDLP summary (all traders including Novo) - previous
for category in ['BTC', 'ETH', 'SOL', 'Alts', 'Passive Beta']:
    category_data = prev_gdlp_summary[prev_gdlp_summary['Category'] == category]
    delta_sum = category_data['$Delta'].sum() if not category_data.empty else 0
    gamma_sum = category_data['$Gamma'].sum() if not category_data.empty else 0
    vega_sum = category_data['$Vega'].sum() if not category_data.empty else 0
    theta_sum = category_data['$Theta'].sum() if not category_data.empty else 0
    
    excel_data[f'Prev GDLP {category} Delta'] = delta_sum
    excel_data[f'Prev GDLP {category} Gamma'] = gamma_sum
    excel_data[f'Prev GDLP {category} Vega'] = vega_sum
    excel_data[f'Prev GDLP {category} Theta'] = theta_sum

# Add GDT data to the excel_data dictionary - current
gdt_key_mapping = {
    'BTC_Delta': 'BTC Net SABR Delta',
    'ETH_Delta': 'ETH Net SABR Delta',
    'SOL_Delta': 'SOL Delta',
    'Alt_Delta': 'Alts Delta',
    'BTC_Gamma': 'BTC Smile Gamma',  
    'ETH_Gamma': 'ETH Smile Gamma',
    'BTC_Vega': 'BTC Vega',
    'ETH_Vega': 'ETH Vega',
    'BTC_Theta': 'BTC Theta',
    'ETH_Theta': 'ETH Theta'
}

# Add current GDT metrics
for gdt_key, excel_key in gdt_key_mapping.items():
    if gdt_key in current_gdt_metrics:
        excel_data[excel_key] = current_gdt_metrics[gdt_key]

# Add previous GDT metrics
for gdt_key, excel_key in gdt_key_mapping.items():
    if gdt_key in prev_gdt_metrics:
        excel_data[f'Prev {excel_key}'] = prev_gdt_metrics[gdt_key]

# Add spot prices
if 'BTC_Spot' in current_gdt_metrics:
    excel_data['BTC Price'] = current_gdt_metrics['BTC_Spot']
if 'ETH_Spot' in current_gdt_metrics:
    excel_data['ETH Price'] = current_gdt_metrics['ETH_Spot']
if 'BTC_Spot' in prev_gdt_metrics:
    excel_data['Prev BTC Price'] = prev_gdt_metrics['BTC_Spot']
if 'ETH_Spot' in prev_gdt_metrics:
    excel_data['Prev ETH Price'] = prev_gdt_metrics['ETH_Spot']

# Convert the dictionary to a DataFrame for creating an Excel sheet
excel_df = pd.DataFrame([excel_data])

# Now let's add this data to the Excel template and map specific cells
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Define the template path
template_path = '../data/new_firmwide_report_template.xlsx'

# Load the Excel template
wb = load_workbook(template_path)

# Print sheet names to identify the correct sheet
print("Available sheets:", wb.sheetnames)

# Create a new sheet for the raw data
raw_data_sheet_name = 'Raw Data'
if raw_data_sheet_name in wb.sheetnames:
    raw_data_sheet = wb[raw_data_sheet_name]
    # Remove existing content
    for row in raw_data_sheet.rows:
        for cell in row:
            cell.value = None
else:
    raw_data_sheet = wb.create_sheet(title=raw_data_sheet_name)

# Write the raw data to the sheet
for r_idx, row in enumerate(dataframe_to_rows(excel_df, index=False, header=True)):
    for c_idx, value in enumerate(row):
        raw_data_sheet.cell(row=r_idx+1, column=c_idx+1, value=value)

# Define cell mappings for current values
delta_mapping = {
    'Novo BTC Delta': 'C4',
    'GDLP ex-Novo BTC Delta': 'C5',
    'BTC Net SABR Delta': 'C6',
    'Novo ETH Delta': 'C8',
    'GDLP ex-Novo ETH Delta': 'C9',
    'ETH Net SABR Delta': 'C10',
    'Novo SOL Delta': 'C12',
    'GDLP ex-Novo SOL Delta': 'C13',
    'SOL Delta': 'C14',
    'Novo Alts Delta': 'C16',
    'GDLP ex-Novo Alts Delta': 'C17',
    'Alts Delta': 'C18',
    'Novo Passive Beta Delta': 'C19'
}

gamma_mapping = {
    'Novo BTC Gamma': 'G4',
    'GDLP ex-Novo BTC Gamma': 'G5',
    'BTC Smile Gamma': 'G6',
    'Novo ETH Gamma': 'G8',
    'GDLP ex-Novo ETH Gamma': 'G9',
    'ETH Smile Gamma': 'G10',
    'Novo SOL Gamma': 'G12',
    'GDLP ex-Novo SOL Gamma': 'G13',
    'Novo Alts Gamma': 'G16',
    'GDLP ex-Novo Alts Gamma': 'G17',
    'Novo Passive Beta Gamma': 'G19'
}

vega_mapping = {
    'Novo BTC Vega': 'K4',
    'GDLP ex-Novo BTC Vega': 'K5',
    'BTC Vega': 'K6',
    'Novo ETH Vega': 'K8',
    'GDLP ex-Novo ETH Vega': 'K9',
    'ETH Vega': 'K10',
    'Novo SOL Vega': 'K12',
    'GDLP ex-Novo SOL Vega': 'K13',
    'Novo Alts Vega': 'K16',
    'GDLP ex-Novo Alts Vega': 'K17',
    'Novo Passive Beta Vega': 'K19'
}

price_mapping = {
    'BTC Price': 'Q3',
    'ETH Price': 'Q4'
}

# Define cell mappings for previous values
prev_delta_mapping = {
    'Prev Novo BTC Delta': 'D4',
    'Prev GDLP ex-Novo BTC Delta': 'D5',
    'Prev BTC Net SABR Delta': 'D6',
    'Prev Novo ETH Delta': 'D8',
    'Prev GDLP ex-Novo ETH Delta': 'D9',
    'Prev ETH Net SABR Delta': 'D10',
    'Prev Novo SOL Delta': 'D12',
    'Prev GDLP ex-Novo SOL Delta': 'D13',
    'Prev SOL Delta': 'D14',
    'Prev Novo Alts Delta': 'D16',
    'Prev GDLP ex-Novo Alts Delta': 'D17',
    'Prev Alts Delta': 'D18',
    'Prev Novo Passive Beta Delta': 'D19'
}

prev_gamma_mapping = {
    'Prev Novo BTC Gamma': 'H4',
    'Prev GDLP ex-Novo BTC Gamma': 'H5',
    'Prev BTC Smile Gamma': 'H6',
    'Prev Novo ETH Gamma': 'H8',
    'Prev GDLP ex-Novo ETH Gamma': 'H9',
    'Prev ETH Smile Gamma': 'H10',
    'Prev Novo SOL Gamma': 'H12',
    'Prev GDLP ex-Novo SOL Gamma': 'H13',
    'Prev Novo Alts Gamma': 'H16',
    'Prev GDLP ex-Novo Alts Gamma': 'H17',
    'Prev Novo Passive Beta Gamma': 'H19'
}

prev_vega_mapping = {
    'Prev Novo BTC Vega': 'L4',
    'Prev GDLP ex-Novo BTC Vega': 'L5',
    'Prev BTC Vega': 'L6',
    'Prev Novo ETH Vega': 'L8',
    'Prev GDLP ex-Novo ETH Vega': 'L9',
    'Prev ETH Vega': 'L10',
    'Prev Novo SOL Vega': 'L12',
    'Prev GDLP ex-Novo SOL Vega': 'L13',
    'Prev Novo Alts Vega': 'L16',
    'Prev GDLP ex-Novo Alts Vega': 'L17',
    'Prev Novo Passive Beta Vega': 'L19'
}

prev_price_mapping = {
    'Prev BTC Price': 'R3',
    'Prev ETH Price': 'R4'
}

# Get the main template sheet (assuming it's the first sheet)
main_sheet = wb[wb.sheetnames[0]]

# Apply the mappings to the template - current values
for column, cell in delta_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

for column, cell in gamma_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

for column, cell in vega_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

for column, cell in price_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

# Apply the mappings to the template - previous values
for column, cell in prev_delta_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

for column, cell in prev_gamma_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

for column, cell in prev_vega_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

for column, cell in prev_price_mapping.items():
    if column in excel_data:
        main_sheet[cell] = excel_data[column]

# Save the workbook with the filled-in data
from datetime import datetime
today = datetime.now().strftime('%Y%m%d')
output_path = f"../output/Firmwide_Report_{today}.xlsx"
wb.save(output_path)

print(f"Excel file saved to: {output_path}")

Available sheets: ['Sheet1']
Excel file saved to: ../output/Firmwide_Report_20250423.xlsx
